##  More than half of all births in Mexico in 2024 were cesarean deliveries


**This project uses the 2024 database from Mexico’s National Birth Certificate Information System (SINAC), compiled by the Ministry of Health, to analyze the prevalence of cesarean deliveries in the context of obstetric saturation.**  
**The analysis focuses on:**
- Quantifying the proportion of cesarean versus vaginal births.  
- Identifying variations by state and by hospital.  
- Evaluating the availability of OB-GYNs at recorded births.  
- Reflecting on the implications for maternal safety and obstetric emergencies.


## Objectives
1. Calculate the national cesarean rate and compare it to vaginal births.  
2. Map the distribution of cesarean deliveries by state and hospital.  
3. Identify hospitals with a high number of births without an OB-GYN present.  
4. Generate inputs for the public debate on maternal health policies in Mexico.


### Libraries:


In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl

### Notebook setup:

In [75]:
pd.set_option('display.max_columns', None)

### Dictionaries

**Diccionario 'orden_columnas' será util para aplicar un orden final al df generado.**

In [76]:
orden_columnas = [
    'CLUES',
    'ENTIDAD_PARTO_CVE',
    'ENTIDAD_PARTO_STR',
    'TIPO_PARTO_CVE',
    'TIPO_PARTO_STR',
    'TIPO_CESAREA_CVE',
    'TIPO_CESAREA_STR',
    'PERSONAL_ATENDIO_CVE',
    'PERSONA_ATENDIO_STR',
    'MEDICO_ATENDIO_CVE',
    'MEDICO_ATENDIO_STR',
    'SOBREVIVIO_PARTO_CVE',
    'SOBREVIVIO_PARTO_STR',
    'FECHA_NACIMIENTO_MADRE'
]


**Diccionario 'medico_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [77]:
medico_map = {
    11: 'MÉDICO GINECO OBSTETRA',
    12: 'OTRO ESPECIALISTA',
    13: 'RESIDENTE',
    14: 'MÉDICO GENERAL',
    15: 'MPSS',
    16: 'MIP',
    9:  'SE IGNORA'
}

**Diccionario 'persona_atendio_parto_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [78]:
personal_atendio_parto_map = {
    0: 'NO ESPECIFICADO',
    1: 'MÉDICO',
    2: 'ENFERMERA',
    3: 'PERSONA AUTORIZADA POR LA SECRETARÍA DE SALUD',
    4: 'PARTERA',
    8: 'OTRO',
    9: 'SE IGNORA' 
}

**Diccionario 'parto_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [79]:
parto_map = {
    0: 'No especificado',
    1: 'Eutócico (natural)',
    2: 'Distócico (natural complicado)',
    3: 'Cesárea',
    8: 'Otro'
}

**Diccionario 'cesareas_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [80]:
cesareas_map = {
    0: "NO ESPECIFICADO",
    1: "PROGRAMADA",
    2: "DE URGENCIA",
    8: "NO APLICA"
}

**Diccionario 'entidadfederativaparto_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [81]:
entidad_federativaparto_map = {
    0: 'NO ESPECIFICADO',
    1: 'AGUASCALIENTES',
    2: 'BAJA CALIFORNIA',
    3: 'BAJA CALIFORNIA SUR',
    4: 'CAMPECHE',
    5: 'COAHUILA DE ZARAGOZA',
    6: 'COLIMA',
    7: 'CHIAPAS',
    8: 'CHIHUAHUA',
    9: 'CIUDAD DE MÉXICO',
    10: 'DURANGO',
    11: 'GUANAJUATO',
    12: 'GUERRERO',
    13: 'HIDALGO',
    14: 'JALISCO',
    15: 'MÉXICO',
    16: 'MICHOACÁN DE OCAMPO',
    17: 'MORELOS',
    18: 'NAYARIT',
    19: 'NUEVO LEÓN',
    20: 'OAXACA',
    21: 'PUEBLA',
    22: 'QUERÉTARO',
    23: 'QUINTANA ROO',
    24: 'SAN LUIS POTOSÍ',
    25: 'SINALOA',
    26: 'SONORA',
    27: 'TABASCO',
    28: 'TAMAULIPAS',
    29: 'TLAXCALA',
    30: 'VERACRUZ DE IGNACIO DE LA LLAVE',
    31: 'YUCATÁN',
    32: 'ZACATECAS',
    88: 'NO APLICA',
    99: 'SE IGNORA'
}

**Diccionario para homologar nombres de entidades***

In [82]:
reemplazos_entidades = {
    "CIUDAD DE MÉXICO": "Ciudad de México",
    "VERACRUZ DE IGNACIO DE LA LLAVE": "Veracruz",
    "MICHOACÁN DE OCAMPO": "Michoacán",
    "COAHUILA DE ZARAGOZA": "Coahuila",
    "BAJA CALIFORNIA SUR": "Baja California Sur",
    "BAJA CALIFORNIA": "Baja California",
    "NUEVO LEÓN": "Nuevo Leon",
    "QUERÉTARO": "Queretaro",
    "SAN LUIS POTOSÍ": "San Luis Potosi",
    "TLAXCALA": "Tlaxcala",
    "SONORA": "Sonora",
    "YUCATÁN": "Yucatan",
    "CAMPECHE": "Campeche",
    "DURANGO": "Durango",
    "ZACATECAS": "Zacatecas",
    "HIDALGO": "Hidalgo",
    "AGUASCALIENTES": "Aguascalientes",
    "MORELOS": "Morelos",
    "QUINTANA ROO": "Quintana Roo",
    "TABASCO": "Tabasco",
    "CHIAPAS": "Chiapas",
    "GUERRERO": "Guerrero",
    "OAXACA": "Oaxaca",
    "NAYARIT": "Nayarit",
    "COLIMA": "Colima",
    "SINALOA": "Sinaloa",
    "JALISCO": "Jalisco",
    "PUEBLA": "Puebla",
    "TAMAULIPAS": "Tamaulipas",
    "CHIHUAHUA": "Chihuahua",
    "ESTADO DE MÉXICO": "Mexico"
}

**Diccionario para identificar si la madre sobrevivió o no al parto (mortalidad materna)**


In [83]:
mortalidad_materna_map = {
    0: "NO ESPECIFICADO",
    1: "SI",
    2: "NO",
    8: "NO APLICA",
    9: "SE IGNORA"
}


### Columnas útiles

In [84]:
columnas_nacimientos = [
    'CLUES',
    'ENTIDADFEDERATIVAPARTO',
    'RESOLUCIONEMBARAZO',
    'TIPOCESAREA',
    'PERSONALATENDIO',
    'TIPOMEDICOATENDIO',
    'SOBREVIVIOPARTO',
    'FECHANACIMIENTOMADRE'
]

In [85]:
columnas_establecimientos_salud = [
    'CLUES',
    'NOMBRE DE LA UNIDAD',
    'LATITUD',
    'LONGITUD',
    'ENTIDAD',
    'CLAVE DE LA ENTIDAD',
    'NOMBRE DE LA INS ADM',
    'NIVEL ATENCION',
    'NOMBRE DE LA INSTITUCION',
    'NOMBRE COMERCIAL',
]


### Data loading:


**Loading the 2024 birth dataset provided by the Ministry of Health**


In [86]:
df_nacimientos_2024 = pd.read_csv("Nacimientos_2024.csv", encoding='utf-8', low_memory=False, usecols=columnas_nacimientos)


**Lectura del catálogo de establecimientos de salud**

In [87]:
df_establecimientos_salud_2024 = pd.read_excel('sinac_catalogos_2024/ESTABLECIMIENTOS_SALUD.xlsx', usecols=columnas_establecimientos_salud)

# Execution:

## Column rename


In [88]:
df_nacimientos_2024 = df_nacimientos_2024.rename(columns={
    'FECHANACIMIENTOMADRE':'FECHA_NACIMIENTO_MADRE',
    'SOBREVIVIOPARTO': 'SOBREVIVIO_PARTO_CVE',
    'CLUES':'CLUES',
    'RESOLUCIONEMBARAZO' : 'TIPO_PARTO_CVE',
    'TIPOCESAREA': 'TIPO_CESAREA_CVE',
    'PERSONALATENDIO':'PERSONAL_ATENDIO_CVE',
    'TIPOMEDICOATENDIO':'MEDICO_ATENDIO_CVE',
    'ENTIDADFEDERATIVAPARTO': 'ENTIDAD_PARTO_CVE',
})

## Converting codes to string format

**Transformación de nombre en columna TIPOMEDICOATENDIO a MEDICO_ATENDIO_STR**

In [89]:
df_nacimientos_2024['MEDICO_ATENDIO_STR'] = df_nacimientos_2024['MEDICO_ATENDIO_CVE'].map(medico_map)

**Transformación de nombre en columna PERSONALATENDIO a PERSONAL_ATENDIO_STR**

In [90]:
df_nacimientos_2024['PERSONA_ATENDIO_STR'] = df_nacimientos_2024['PERSONAL_ATENDIO_CVE'].map(personal_atendio_parto_map)

**Transformación de nombre en columna RESOLUCIONEMBARAZO a TIPO_PARTO_STR**

In [91]:
df_nacimientos_2024['TIPO_PARTO_STR'] = df_nacimientos_2024['TIPO_PARTO_CVE'].map(parto_map)

**Transformación de nombre en columna ENTIDADFEDERATIVAPARTO a ENTIDAD_PARTO_STR**

In [92]:
df_nacimientos_2024['ENTIDAD_PARTO_STR'] = df_nacimientos_2024['ENTIDAD_PARTO_CVE'].map(entidad_federativaparto_map)

**Transformación de nombre en columna SOBREVIVIOPARTO a SOBREVIVIO_PARTO_CVE**

In [93]:
df_nacimientos_2024['SOBREVIVIO_PARTO_STR'] = df_nacimientos_2024['SOBREVIVIO_PARTO_CVE'].map(mortalidad_materna_map)

**Transformación de nombre en columna TIPO_CESAREA_CVE a TIPO_CESAREA_STR**

In [94]:
df_nacimientos_2024['TIPO_CESAREA_STR'] = df_nacimientos_2024['TIPO_CESAREA_CVE'].map(cesareas_map)

## Column rename

In [95]:
df_nacimientos_2024 = df_nacimientos_2024[orden_columnas]

## Methodological note

#### The original dataset recorded a total of 1,413,203 births. First, 28 null (NaN) records in the CLUES column were removed, leaving 1,413,175. Then, by excluding records with the value '9998', the dataset was reduced to 1,338,905 births with complete information about the medical facility.


## Splitting the __df_nacimientos_2024__ dataframe based on available data


**DataFrame de nacimientos 2024 íntegro sin modificaciones** (copia la referencia (en pandas esto no duplica memoria), y muestra el total de filas iniciales)

In [96]:
# 1) Partimos de tu DataFrame “base” (antes de limpiar nada):
df_nacimientos_2024 = df_nacimientos_2024
len(df_nacimientos_2024)  
# → 1 413 203 registros totales

1413203

In [97]:
df_nacimientos_2024['CLUES'].isna().sum()

np.int64(28)

**DataFrame de nacimientos 2024, sin valores NaN en la columna 'CLUES'**

In [98]:
df_nacimientos_2024_clean_clues = df_nacimientos_2024.dropna(subset=['CLUES'])
len(df_nacimientos_2024_clean_clues)

1413175

**DataFrame de nacimientos 2024, sin valores 9998 en la columna 'CLUES'**

In [99]:
# 4) Ahora quitamos también los registros con CLUES == '9998'
#    (código que usas para marcarlas como “sin identificador oficial”)
df_nacimientos_2024_clean_clues = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues['CLUES'] != '9998'
]
len(df_nacimientos_2024_clean_clues)
# → 1 338 905 registros

1338905

In [100]:
# 5) Reordenas las columnas según tu esquema final
df_nacimientos_2024_clean_clues = df_nacimientos_2024_clean_clues[orden_columnas]

## How many births were registered in 2024, and how were they distributed by type of delivery?


**Determinar el número de registros en df_nacimientos_2024 que contiene la totalidad de nacimientos en México en el año 2024**

In [101]:
total_nacimientos = len(df_nacimientos_2024)
print("El total de nacimientos en 2024 fue:",total_nacimientos)

El total de nacimientos en 2024 fue: 1413203


### Cesarean births in Mexico 2024: far above the WHO standard

In 2024, as shown below, 56.1% of all births were cesarean deliveries.

The World Health Organization (WHO) recommends an ideal rate between **10% and 15%**, only when medically necessary.

Such a high rate may indicate **hospital overcrowding, lack of specialized personnel, or unnecessary interventions**.


**Count the number of births by type of delivery**


In [102]:
df_distribucion_tipo_parto = (
    df_nacimientos_2024["TIPO_PARTO_STR"]
    .value_counts(dropna=False)                  
    .rename_axis("TIPO_PARTO")
    .reset_index(name="TOTAL")
    .assign(PORCENTAJE=lambda d: (d["TOTAL"] / total_nacimientos * 100).round(1))
    .sort_values("PORCENTAJE", ascending=False)
    .reset_index(drop=True)
)

**Generación .csv de df_distribucion_tipo_parto**

In [103]:
df_distribucion_tipo_parto.to_csv('tipo_parto_distribucion_2024_plot.csv', index=False)

## Cesarean deliveries by state







1. A binary column `ES_CESAREA` was created based on `TIPO_PARTO_CVE == 3`.  
2. Data were grouped by state (`ENTIDAD_PARTO_STR`) to count:  
   - The total number of births (`TOTAL_PARTOS`).  
   - The total number of cesarean deliveries (`TOTAL_CESAREAS`).  
3. A national ranking was generated, sorted from highest to lowest percentage.


In [104]:
df_nacimientos_2024["ES_CESAREA"] = df_nacimientos_2024["TIPO_PARTO_CVE"] == 3

In [105]:
df_cesareas_por_estado["ENTIDAD_PARTO_STR"] = df_cesareas_por_estado["ENTIDAD_PARTO_STR"].replace(reemplazos_entidades)

NameError: name 'df_cesareas_por_estado' is not defined

In [ ]:
df_cesareas_por_estado["PORCENTAJE_CESAREAS"] = (
    df_cesareas_por_estado["TOTAL_CESAREAS"] / df_cesareas_por_estado["TOTAL_PARTOS"] * 100
).round(1)

In [ ]:
df_cesareas_por_estado = df_cesareas_por_estado.sort_values("PORCENTAJE_CESAREAS", ascending=False).reset_index(drop=True)
df_cesareas_por_estado["RANK"] = df_cesareas_por_estado.index + 1

In [ ]:
df_cesareas_por_estado["RANK"] = df_cesareas_por_estado.index + 1

In [ ]:
df_cesareas_por_estado.head(10)

In [ ]:
df_export = df_cesareas_por_estado[["ENTIDAD_PARTO_STR", "PORCENTAJE_CESAREAS"]].rename(
    columns={
        "ENTIDAD_PARTO_STR": "Entidad",
        "PORCENTAJE_CESAREAS": "Porcentaje"
    }
)

In [ ]:
df_export.to_csv("porcentaje_cesareas_por_estado_plot.csv", index=False)

In [ ]:
# Suma total nacional
total_partos_nacional = df_cesareas_por_estado["TOTAL_PARTOS"].sum()
total_cesareas_nacional = df_cesareas_por_estado["TOTAL_CESAREAS"].sum()

In [ ]:
print(total_partos_nacional)
print(total_cesareas_nacional)


## Identify births by state, hospitals (CLUES), and type of cesarean delivery.

**The original dataset contains 28,885 records where the variable MEDICO_ATENDIO (CVE, STR) appears as NaN, meaning the type of medical professional who attended those births was not reported.**


In [ ]:
missing_counts = df_nacimientos_2024.isna().sum()

missing_pct = (missing_counts / len(df_nacimientos_2024) * 100).round(2)

report = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_pct": missing_pct
}).loc[missing_counts > 0].sort_values("missing_count", ascending=False)


In [ ]:
#ORDENAR POR CLUES
df_nacimientos_2024 = df_nacimientos_2024.sort_values(by='CLUES')

In [ ]:
#TEMP
len(df_nacimientos_2024)

In [ ]:
df_nacimientos_2024.to_csv('dataset_nacimientos_2024.csv', index=False)

## Top 10 hospitals with the most births without OB-GYNs and facility level


In [ ]:
# df_nacimientos_2024_clean_clues

df_top_ranking_hospitales = df_nacimientos_2024_clean_clues.copy()

In [ ]:
#Filtrar los que fueron atentidos por un gineco-obstetra. 
df_top_ranking_hospitales['ES_GINECO'] = df_nacimientos_2024_clean_clues['MEDICO_ATENDIO_STR'].str.contains('MÉDICO GINECO OBSTETRA', case=False, na=False)

In [ ]:
df_top_ranking_hospitales.head()

In [ ]:
df_top_ranking_hospitales = (
    df_top_ranking_hospitales.groupby('CLUES')
    .agg(
        TOTAL_PARTOS=('TIPO_PARTO_STR', 'count'),
        PARTOS_CON_GINECO=('ES_GINECO', 'sum')
    )
    .reset_index()
)

df_top_ranking_hospitales['PARTOS_SIN_GINECO'] = df_top_ranking_hospitales['TOTAL_PARTOS'] - df_top_ranking_hospitales['PARTOS_CON_GINECO']

**Pegar metadatos**

In [ ]:
df_tabla_hospitales = df_top_ranking_hospitales[[
    'ENTIDAD',
    'NOMBRE COMERCIAL',
    'TOTAL_PARTOS',
    'PARTOS_SIN_GINECO',
    'TASA_SIN_GINECO',
    'NIVEL ATENCION'
]]

In [ ]:
# 2. Ordenamos por partos sin gineco y nos quedamos con el top 10
df_tabla_hospitales = df_tabla_hospitales.sort_values(
    by='PARTOS_SIN_GINECO', ascending=False
).head(10)

In [ ]:
# 3. Renombramos las columnas al inglés para Datawrapper
df_tabla_hospitales = df_tabla_hospitales.rename(columns={
    'ENTIDAD': 'State',
    'NOMBRE COMERCIAL': 'Hospital Name',
    'TOTAL_PARTOS': 'Total Births',
    'PARTOS_SIN_GINECO': 'Births Without OB-GYN',
    'TASA_SIN_GINECO': 'Percent Without OB-GYN',
    'NIVEL ATENCION': 'Level of Care'
})

In [ ]:
df_tabla_hospitales.to_csv("top_10_hospitals_obgyn_gap_en.csv", index=False)


In [ ]:
df_tabla_hospitales["Percent Without OB-GYN"] = df_tabla_hospitales["Percent Without OB-GYN"].apply(lambda x: f"{x:.2f}")

In [ ]:
df_tabla_hospitales.to_csv("top_10_hospitals_obgyn_gap_en.csv", index=False)

# Types of cesarean deliveries in Mexico


## Analysis of cesarean delivery types in Mexico (2024)

Based on the cleaned DataFrame (`df_nacimientos_2024_clean_clues`), we will:
1. Filter only the births that were cesarean deliveries  
2. Count all cesarean categories  
3. Build a DataFrame with totals and percentages  
4. Export the CSV


**Filter only births that were cesarean deliveries**

In [ ]:
df_ces_clean = df_nacimientos_2024_clean_clues[
  df_nacimientos_2024_clean_clues['TIPO_PARTO_STR']=='Cesárea'
]

**Count all cesarean categories (including NOT SPECIFIED, NOT APPLICABLE…)**


In [ ]:
counts_clean = (
  df_ces_clean['TIPO_CESAREA_CVE']
    .map(cesareas_map)
    .value_counts()
)

**Construir el DataFrame final con totales y porcentaje**


In [ ]:
pct_urg_clean = round(counts_clean['DE URGENCIA'] / counts_clean.sum() * 100, 1)
print(f"{pct_urg_clean}% de las cesáreas fueron de urgencia.")

In [ ]:
df_ces_clean = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues['TIPO_PARTO_STR']=='Cesárea'
]

In [ ]:
counts_clean = (
    df_ces_clean['TIPO_CESAREA_CVE']
    .map(cesareas_map)
    .value_counts()
)

**Este CSV tendrá columnas `TIPO_CESAREA`, `TOTAL_CESAREAS` y `PORCENTAJE`**


In [ ]:
df_plot = counts_clean.rename_axis('TIPO_CESAREA') \
                      .reset_index(name='TOTAL_CESAREAS')
df_plot['PORCENTAJE'] = (
    df_plot['TOTAL_CESAREAS'] / df_plot['TOTAL_CESAREAS'].sum() * 100
).round(1)


In [ ]:
df_plot

**Guardar CSV listo para plot**

In [ ]:
df_plot.to_csv('tipos_cesareas_mexico_2024_plot.csv', index=False)

## Maternal mortality




**How many births in df_nacimientos_2024 resulted in the mother's death?**

**Extract only the relevant columns for the maternal death analysis by creating a copy of `df_nacimientos_2024_clean_clues`, in order to avoid modifying the original dataframe**


In [ ]:
df_mortalidad_materna_2024 = df_nacimientos_2024_clean_clues[[
    'CLUES',
    'SOBREVIVIO_PARTO_CVE',
    'SOBREVIVIO_PARTO_STR',
    'ENTIDAD_PARTO_STR'
]].copy()   

In [ ]:
df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] = (
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE']
    .astype(str)
)

**Filtrar los casos de muerte materna (SOBREVIVIO_PARTO_CVE == '2')**


In [ ]:
df_madre_no_sobrevivio = df_mortalidad_materna_2024[
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] == '2'
]

**Filtrar los casos donde la madre sobrevivió (SOBREVIVIO_PARTO_CVE == '1')**

In [ ]:
df_madre_sobrevivio = df_mortalidad_materna_2024[
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] == '1'
]

**Filtrar los casos “no especificado” (SOBREVIVIO_PARTO_CVE == '0')**

In [ ]:
df_madre_no_especificado = df_mortalidad_materna_2024[
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] == '0'
]

**Resultados de la operación sobre supervivencia de las madres**

In [ ]:
print(f"According to the birth dataset, only {len(df_madre_no_sobrevivio)} maternal deaths were reported in 2024.")
print(f"It was reported that in {len(df_madre_sobrevivio)} births, the mother survived.")
print(f"There are {len(df_madre_no_especificado)} cases where it is not specified whether the mother survived childbirth or not.")


## What type of medical professional attended the births?


To ensure that **all** maternal deaths are counted (including those without a reported medical provider), we follow these steps:

1. **Filter** maternal deaths (`SOBREVIVIO_PARTO_CVE == 2`) from the cleaned DataFrame.  
2. **Count** by code (`MEDICO_ATENDIO_CVE`) and **map** to readable labels.  
3. Explicitly include `NaN` values using `dropna=False` to avoid losing any cases.  
4. **Calculate** the percentage out of the total number of deaths.  
5. **Verify** that the sum of `count` matches the total number of deaths and that the percentages add up to 100%.


**Filtrar muertes maternas (código entero 2)**

In [ ]:
df_madre_no_sobrevivio = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues["SOBREVIVIO_PARTO_CVE"] == 2
]

**Contar por código y luego mapear etiquetas (incluye NaN)**

In [ ]:
conteo_medicos = (
    df_madre_no_sobrevivio["MEDICO_ATENDIO_CVE"]
    .map(medico_map)
    .value_counts(dropna=False)      # incluye los NaN
    .rename_axis("TIPO_MEDICO")
    .reset_index(name="cantidad")
)

**Calcular porcentaje**

In [ ]:
total = conteo_medicos["cantidad"].sum()
conteo_medicos["porcentaje"] = (conteo_medicos["cantidad"] / total * 100).round(1)

In [ ]:
conteo_medicos.to_csv("tipos_medico_muerte_materna_plot.csv", index=False)

# Which hospitals reported the most maternal deaths?

In [ ]:
#VERIFICAR LOS UNIQUE EN EL DF DE MUERTES MATERNAS

hospitales_muertes_maternas = df_madre_no_sobrevivio['CLUES'].unique()

#CONTAR LOS REPETIDOS

df_clues_repetidos = df_madre_no_sobrevivio['CLUES'].value_counts()

#CONVERTIR DE SERIE A DF PARA PODER COMPARAR EN EL FUTURO

df_clues_repetidos = df_clues_repetidos.to_frame()

print(type(df_clues_repetidos))

df_clues_con_nombre = pd.merge(
    df_clues_repetidos,
    df_establecimientos_salud_2024[['CLUES', 'NOMBRE DE LA UNIDAD','NOMBRE DE LA INS ADM','ENTIDAD', 'LATITUD', 'LONGITUD']],
    on='CLUES',
    how='left'
)

df_clues_con_nombre.head()


In [ ]:
#Generar .csv de los hospitales con más de 2 registros con muerte materna
df_semaforo_gineco.to_csv('hospitales_muertes_materna.csv', index=False)

**In the 2024 birth dataset, 76 maternal deaths were reported, along with another 385 cases classified as “not specified.” It is necessary to investigate which hospitals reported these cases to better understand the conditions of care and the gaps in recordkeeping.**


In [ ]:
def clasificar_parto_detallado(tipo_parto_str, tipo_cesarea_str):
    if tipo_parto_str == "Cesárea":
        if tipo_cesarea_str == "PROGRAMADA":
            return "Cesárea programada"
        elif tipo_cesarea_str == "DE URGENCIA":
            return "Cesárea de urgencia"
        else:
            return "Cesárea no especificada"
    elif tipo_parto_str == "Eutócico (natural)":
        return "Eutócico - Natural"
    elif tipo_parto_str == "Distócico (natural complicado)":
        return "Distócico - Natural Complicao"
    elif tipo_parto_str == "Otro":
        return "Otro"
    else:
        return "Parto No especificado"


In [ ]:
df_nacimientos_2024_clean_clues["PARTO_DETALLADO"] = df_nacimientos_2024_clean_clues.apply(
    lambda row: clasificar_parto_detallado(row["TIPO_PARTO_STR"], row["TIPO_CESAREA_STR"]),
    axis=1
)

In [ ]:
df_nacimientos_2024_clean_clues["PARTO_DETALLADO"].value_counts()

In [ ]:
df_muertes_detallado = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues['SOBREVIVIO_PARTO_CVE'] == 2
].copy()

## In which type of delivery did most maternal deaths occur?

Of the 76 maternal deaths recorded in the 2024 birth dataset, nearly half occurred after an emergency cesarean (48.7%), making it the type of delivery with the highest recorded lethality.

A total of 30.3% of the deaths followed a natural (eutocic) vaginal birth, and 17.1% occurred after a scheduled cesarean, indicating that even non-emergency procedures can carry risks if not performed under proper conditions.

Only one case was reported as “unspecified cesarean,” which also raises concerns about inconsistencies or gaps in obstetric recordkeeping.



In [ ]:
conteo_muertes = (
    df_muertes_detallado["PARTO_DETALLADO"]
    .value_counts()
    .rename_axis("TIPO_PARTO_MUERTE_MATERNA")
    .reset_index(name="Cantidad")
)

In [ ]:
total_muertes = conteo_muertes["Cantidad"].sum()
conteo_muertes["Porcentaje"] = round((conteo_muertes["Cantidad"] / total_muertes) * 100, 1)

In [ ]:
conteo_muertes

In [ ]:
conteo_muertes.to_csv("muertes_maternas_por_parto_detallado.csv", index=False)